# Formatting Data for LSTM

In [87]:
import pandas as pd
import numpy as np
import time
from timeit import default_timer as timer
import pickle

memory_len = 10

In [91]:

def format_data(updated_data,frequency,memory_len = memory_len  ):
    df = pd.DataFrame(updated_data,columns=['price','datetime'])
    df.datetime =df.datetime
    df.datetime = pd.to_datetime(df.datetime)
    df = df.set_index('datetime')

    # Populate Group Indicators
    def nth(listt,k):
        try:
            return listt[k]
        except:
            return()
    base=df.index.max()-pd.Timedelta(hours=100000)
    
    df = df.groupby(pd.Grouper(freq='5min',origin=base,closed='right')).agg(
        open_price = pd.NamedAgg(column='price',aggfunc=lambda x: nth(x,0)),
        high=pd.NamedAgg(column='price',aggfunc=max),
        low=pd.NamedAgg(column='price',aggfunc=min),
        close_price = pd.NamedAgg(column='price',aggfunc=lambda x: nth(x,-1)),
        tradecount=pd.NamedAgg(column='price',aggfunc=len),
  #      mean=pd.NamedAgg(column='price',aggfunc=np.mean),
  #      median=pd.NamedAgg(column='price',aggfunc=np.median),
        

        )
    
    # we need to throw out the running start
    min30=[0]*memory_len 
    for i in range(memory_len,df.shape[0]):
        min30.append(max(df.high[i-memory_len:i]))
    df['high30min'] = min30
    df= df[memory_len+1:]

    # apply scaler transform
    scaler=pickle.load(open('lstm2_2month_30minfuture_rolling_10_blocks_scalar.pkl','rb'))
    dummy = scaler.transform(df)
    
    # block the data for prediction
    lstm_data=[]
    for i in range(memory_len , dummy.shape[0]):
        lstm_data.append(dummy[i-memory_len:i])
    lstm_data= np.array(lstm_data)
    return lstm_data

In [76]:
data = pd.read_csv('/Users/kyleevans-lee/proj/notebooks/crypto/Binance_BTCUSDT_minute.csv')
memory_len = 10

In [77]:
ld = data[['open','date']].values[:1000]


In [110]:
start = timer()
test = format_data(ld,'1m')
end = timer()
print(end - start) 


0.1022764740000639


In [112]:
test.shape

(179, 10, 6)

In [117]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow
model = tensorflow.keras.models.load_model('lstm10hour_5features/')
scaler=pickle.load(open('lstm2_2month_30minfuture_rolling_10_blocks_scalar.pkl','rb'))
    
#dataframe['lstm_predicted_high_30_min'] = dataframe['median'].ewm(span=2,min_periods=2,adjust=False).mean()


In [124]:
Y_pred_unscaled=model.predict(test)
Y_pred=np.append([0]*memory_len,scaler.inverse_transform(
    [list(np.append([0]*5,x)) for x in Y_pred_unscaled])[:,5])

In [125]:
len(Y_pred)

189

(960180, 12)

In [ ]:

test.index.max().asm8//

In [ ]:
seconds = test.index.max().second
minutes = pd.Timedelta('5min').seconds/60

In [ ]:
base=test.index.max()-pd.Timedelta(hours=100000)

In [ ]:
base

In [ ]:
df =test
grps = (df.index.min()-df.index.max()-pd.Timedelta(days=1)).dt.floor('30D')
df.groupby(grps).agg({'date': 'last', 'a': 'sum'})

In [ ]:
test.groupby(.size()

In [ ]:
'open', 'high', 'low', 'close','tradecount', 'high30min'

In [ ]:
df = format_data(ld[:1001],'1min')


In [ ]:
df

In [ ]:
min30=[0]*memory_len 
for i in range(memory_len,df.shape[0]):
    print(i)
    print(df.high[i-memory_len:i])
    min30.append(max(data.high[i-30:i]))
    break
df['high30min'] = min30

In [ ]:

df = Strategy.format_data(updated_data=start_data,frequency=frequency)
df = Strategy.populate_indicators(dataframe = df)
df = Strategy.populate_buy_trend(dataframe=df)